In [ ]:
import json

# Read the metadata from a JSONLD file
with open("workflows-bioschemas-dump (1).jsonld", "r", encoding="utf-8") as f:
    data = json.load(f)

# Extract all items where the programmingLanguage["@id"] field is "#cwl"
cwl_items = [
    item for item in data
    if "programmingLanguage" in item  # Check that the item has a programmingLanguage field
    and isinstance(item["programmingLanguage"], dict)  # Ensure it's a dictionary (not a list or string)
    and item["programmingLanguage"].get("@id") == "#cwl"  # Check if the language is CWL (Common Workflow Language)
]

# Print the number of CWL workflows found
print(f"Found {len(cwl_items)} items where programmingLanguage['@id'] == '#cwl'")

# Optionally print the first two items for inspection (currently commented out)
# print(json.dumps(cwl_items[:2], indent=2))

# Print ID, name, and URL of each CWL workflow
for item in cwl_items:
    print(f"ID: {item.get('@id', 'N/A')}")        # Print the workflow ID (or 'N/A' if missing)
    print(f"Name: {item.get('name', 'N/A')}")     # Print the workflow name
    print(f"URL: {item.get('url', 'N/A')}")       # Print the workflow URL
    print("--------")                             # Print a separator line

In [ ]:
from rdflib import Graph, Namespace
from rdflib.namespace import RDF


g = Graph()
g.parse("workflowhub-graph.ttl", format="ttl")  


SCHEMA = Namespace("http://schema.org/")


query = """
PREFIX schema1: <http://schema.org/>

SELECT ?s ?lang ?name ?id WHERE {
  ?s schema1:programmingLanguage ?lang .
  ?s schema1:name ?name . 
  ?s schema1:url ?id . 
  FILTER(CONTAINS(STR(?lang), "#cwl"))
}
"""

results = list(g.query(query))  

print(f"Total matched results: {len(results)}")
workflow_info = []
for row in results:
    workflow_info.append({
        "name": str(row.name),
        "id": str(row.id)
    })
print(workflow_info[:2])  

Total matched results: 74
[{'name': 'Assemblosis', 'id': 'https://workflowhub.eu/workflows/334?version=1'}, {'name': 'Indices builder from GBOL RDF (TTL)', 'id': 'https://workflowhub.eu/workflows/75?version=1'}]


In [ ]:
import requests
import re
import json
from tqdm import tqdm

def extract_id_and_version(url):
    match = re.search(r'/workflows/(\d+)\?version=(\d+)', url)
    if match:
        return match.group(1), match.group(2)
    return None, None

all_workflows_data = []

for entry in tqdm(workflow_info):
    workflow_id, version = extract_id_and_version(entry["id"])
    if not workflow_id or not version:
        continue

    print(f"\n🔍 Processing workflow {workflow_id} (version {version})")
    base_url = f"https://workflowhub.eu/ga4gh/trs/v2/tools/{workflow_id}/versions/{version}/cwl"
    files_url = f"{base_url}/files"
    print(f"📄 Fetching file list: {files_url}")

    try:
        files_resp = requests.get(files_url, timeout=10)
        files_resp.raise_for_status()
        file_list = files_resp.json()

        file_contents = []
        for file in file_list:
            path = file["path"]
            file_type = file.get("file_type", "")
            desc_url = f"{base_url}/descriptor/{path}"
            print(f"📥 Fetching descriptor: {desc_url}")

            try:
                content_resp = requests.get(desc_url, timeout=10)
                content_resp.raise_for_status()
                content = content_resp.text
            except requests.exceptions.Timeout:
                print(f"⏱️ Timeout fetching descriptor {path}")
                content = None
            except requests.exceptions.RequestException as e:
                print(f"❌ Error fetching descriptor {path}: {e}")
                content = None

            file_contents.append({
                "path": path,
                "file_type": file_type,
                "content": content
            })

        all_workflows_data.append({
            "workflow_id": workflow_id,
            "version": version,
            "name": entry.get("name", ""),
            "files": file_contents
        })

    except requests.exceptions.Timeout:
        print(f"⏱️ Timeout fetching file list")
    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching file list: {e}")
    except Exception as e:
        print(f"❌ Unexpected error: {e}")


with open("workflow_files_dump.json", "w", encoding="utf-8") as f:
    json.dump(all_workflows_data, f, indent=2, ensure_ascii=False)

  0%|          | 0/74 [00:00<?, ?it/s]


🔍 Processing workflow 334 (version 1)
📄 Fetching file list: https://workflowhub.eu/ga4gh/trs/v2/tools/334/versions/1/cwl/files
📥 Fetching descriptor: https://workflowhub.eu/ga4gh/trs/v2/tools/334/versions/1/cwl/descriptor/Examples/assemblyCele.yml
📥 Fetching descriptor: https://workflowhub.eu/ga4gh/trs/v2/tools/334/versions/1/cwl/descriptor/Examples/assemblyDmel.yml
📥 Fetching descriptor: https://workflowhub.eu/ga4gh/trs/v2/tools/334/versions/1/cwl/descriptor/Examples/assemblyPfal.yml
📥 Fetching descriptor: https://workflowhub.eu/ga4gh/trs/v2/tools/334/versions/1/cwl/descriptor/Examples/metricsCele.yml
📥 Fetching descriptor: https://workflowhub.eu/ga4gh/trs/v2/tools/334/versions/1/cwl/descriptor/Examples/metricsDmel.yml
📥 Fetching descriptor: https://workflowhub.eu/ga4gh/trs/v2/tools/334/versions/1/cwl/descriptor/Examples/metricsPfal.yml
📥 Fetching descriptor: https://workflowhub.eu/ga4gh/trs/v2/tools/334/versions/1/cwl/descriptor/LICENSE
📥 Fetching descriptor: https://workflowhub.eu/

  1%|▏         | 1/74 [00:42<51:12, 42.09s/it]

❌ Error fetching descriptor workflow-334-c498607f21124d2f3787a21c0d554375799170a7-diagram.svg: 404 Client Error: Not Found for url: https://workflowhub.eu/ga4gh/trs/v2/tools/334/versions/1/cwl/descriptor/workflow-334-c498607f21124d2f3787a21c0d554375799170a7-diagram.svg

🔍 Processing workflow 75 (version 1)
📄 Fetching file list: https://workflowhub.eu/ga4gh/trs/v2/tools/75/versions/1/cwl/files
📥 Fetching descriptor: https://workflowhub.eu/ga4gh/trs/v2/tools/75/versions/1/cwl/descriptor/workflow_indexbuilder.cwl
📥 Fetching descriptor: https://workflowhub.eu/ga4gh/trs/v2/tools/75/versions/1/cwl/descriptor/ro-crate-metadata.json
📥 Fetching descriptor: https://workflowhub.eu/ga4gh/trs/v2/tools/75/versions/1/cwl/descriptor/ro-crate-preview.html
📥 Fetching descriptor: https://workflowhub.eu/ga4gh/trs/v2/tools/75/versions/1/cwl/descriptor/workflow-75-f8d3db7af24d8ad8fceac69baf83f572a64abff2-diagram.svg


  3%|▎         | 2/74 [00:43<21:42, 18.10s/it]

...

❌ Error fetching descriptor workflow-524-a80a6c7a268fd71886a255fc45afa882d80bda0c-diagram.svg: 404 Client Error: Not Found for url: https://workflowhub.eu/ga4gh/trs/v2/tools/524/versions/1/cwl/descriptor/workflow-524-a80a6c7a268fd71886a255fc45afa882d80bda0c-diagram.svg

🔍 Processing workflow 1253 (version 1)
📄 Fetching file list: https://workflowhub.eu/ga4gh/trs/v2/tools/1253/versions/1/cwl/files
📥 Fetching descriptor: https://workflowhub.eu/ga4gh/trs/v2/tools/1253/versions/1/cwl/descriptor/workflow.cwl
📥 Fetching descriptor: https://workflowhub.eu/ga4gh/trs/v2/tools/1253/versions/1/cwl/descriptor/ro-crate-metadata.json
📥 Fetching descriptor: https://workflowhub.eu/ga4gh/trs/v2/tools/1253/versions/1/cwl/descriptor/ro-crate-preview.html
📥 Fetching descriptor: https://workflowhub.eu/ga4gh/trs/v2/tools/1253/versions/1/cwl/descriptor/workflow-1253-d403ac5aa36b93742b9408a438b12c3436b24bfd-diagram.svg


 97%|█████████▋| 72/74 [2:08:07<01:22, 41.12s/it]

❌ Error fetching descriptor workflow-1253-d403ac5aa36b93742b9408a438b12c3436b24bfd-diagram.svg: 404 Client Error: Not Found for url: https://workflowhub.eu/ga4gh/trs/v2/tools/1253/versions/1/cwl/descriptor/workflow-1253-d403ac5aa36b93742b9408a438b12c3436b24bfd-diagram.svg

🔍 Processing workflow 1206 (version 1)
📄 Fetching file list: https://workflowhub.eu/ga4gh/trs/v2/tools/1206/versions/1/cwl/files
📥 Fetching descriptor: https://workflowhub.eu/ga4gh/trs/v2/tools/1206/versions/1/cwl/descriptor/.cursorignore
📥 Fetching descriptor: https://workflowhub.eu/ga4gh/trs/v2/tools/1206/versions/1/cwl/descriptor/.devcontainer/Dockerfile
📥 Fetching descriptor: https://workflowhub.eu/ga4gh/trs/v2/tools/1206/versions/1/cwl/descriptor/.devcontainer/devcontainer.json
📥 Fetching descriptor: https://workflowhub.eu/ga4gh/trs/v2/tools/1206/versions/1/cwl/descriptor/.gitignore
📥 Fetching descriptor: https://workflowhub.eu/ga4gh/trs/v2/tools/1206/versions/1/cwl/descriptor/LICENSE
📥 Fetching descriptor: http

100%|██████████| 74/74 [2:31:42<00:00, 123.00s/it]

❌ Error fetching descriptor workflow-1206-b8c0b1de1814cb4f0f6693c45ad2a979bfe160da-diagram.svg: 404 Client Error: Not Found for url: https://workflowhub.eu/ga4gh/trs/v2/tools/1206/versions/1/cwl/descriptor/workflow-1206-b8c0b1de1814cb4f0f6693c45ad2a979bfe160da-diagram.svg


In [1]:
import json

# 加载两个 JSON 文件
with open("workflow_files_dump.json", "r", encoding="utf-8") as f:
    workflow_data = json.load(f)

with open("workflows-bioschemas-dump (1).jsonld", "r", encoding="utf-8") as f:
    metadata_list = json.load(f)

# 创建一个快速查询字典：@id => metadata对象
metadata_map = {
    item["@id"]: item
    for item in metadata_list
    if "@id" in item
}

# 遍历 workflow 数据，匹配并加入元数据
for workflow in workflow_data:
    workflow_id = workflow.get("workflow_id")
    if not workflow_id:
        continue
    workflow_url = f"https://workflowhub.eu/workflows/{workflow_id}"

    matched_metadata = metadata_map.get(workflow_url)
    if matched_metadata:
        workflow["metadata"] = matched_metadata
    else:
        print(f"⚠️ No metadata found for workflow_id: {workflow_id}")

# 保存合并后的结果
with open("workflow_files_with_metadata.json", "w", encoding="utf-8") as f:
    json.dump(workflow_data, f, indent=2, ensure_ascii=False)

⚠️ No metadata found for workflow_id: 1728
